<a href="https://colab.research.google.com/github/RogerHeederer/PyTorchNLPBook/blob/master/SentimentClassfi_yelpLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Minimally clean teh data and derive final dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
import collections
import numpy as np
import pandas as pd
import re

from argparse import Namespace

In [51]:
args = Namespace(
    raw_train_dataset_csv = "/content/drive/MyDrive/RogerHeederer/NLPwithPytorch_B/data/dataset/yelp/raw_train.csv",
    raw_test_dataset_csv = "/content/drive/MyDrive/RogerHeederer/NLPwithPytorch_B/data/dataset/yelp/raw_test.csv",
    proportion_subset_of_train = 0.1,
    train_proportion = 0.7,
    val_proportion=0.15,
    test_proportion=0.15,
    output_munged_csv="/content/drive/MyDrive/RogerHeederer/NLPwithPytorch_B/data/dataset/yelp/reviews_with_splits_lite.csv",
    seed=1337
)

In [5]:
# raw data 읽어오기
train_reviews = pd.read_csv(args.raw_train_dataset_csv, header=None, names=['rating', 'review'])

** Lite 데이터 셋에서는 Rating 1은 부정, 2는 긍정 **

** FULL 데이터 셋에서는 0,1은 부정. 2,3는 긍정 **

In [6]:
train_reviews[0:10]

,rating,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...
5,1,Wing sauce is like water. Pretty much a lot of...
6,1,Owning a driving range inside the city limits ...
7,1,This place is absolute garbage... Half of the...
8,2,Before I finally made it over to this range I ...
9,2,I drove by yesterday to get a sneak peak. It ...


In [7]:
by_rating = collections.defaultdict(list)
by_rating

defaultdict(list, {})

In [8]:
# rating을 key값으로 묶어서, review를 붙여버림
for _, row in train_reviews.iterrows():
    by_rating[row.rating].append(row.to_dict())

In [9]:
by_rating[1][0:5]

[{'rating': 1,
  'review': "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars."},
 {'rating': 1,
  'review': "I don't know what Dr. Goldberg was like before  moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in the co-pay and having you com

In [11]:
by_rating[2][0:5]

[{'rating': 2,
  'review': "Been going to Dr. Goldberg for over 10 years. I think I was one of his 1st patients when he started at MHMG. He's been great over the years and is really all about the big picture. It is because of him, not my now former gyn Dr. Markoff, that I found out I have fibroids. He explores all options with you and is very patient and understanding. He doesn't judge and asks all the right questions. Very thorough and wants to be kept in the loop on every aspect of your medical health and your life."},
 {'rating': 2,
  'review': 'All the food is great here. But the best thing they have is their wings. Their wings are simply fantastic!!  The \\"Wet Cajun\\" are by the best & most popular.  I also like the seasoned salt wings.  Wing Night is Monday & Wednesday night, $0.75 whole wings!\\n\\nThe dining area is nice. Very family friendly! The bar is very nice is well.  This place is truly a Yinzer\'s dream!!  \\"Pittsburgh Dad\\" would love this place n\'at!!'},
 {'ratin

In [10]:
by_rating.keys()

dict_keys([1, 2])

In [16]:
review_subset = []
#dict.items()는 key와 value를 _, item_list로 올려줌.
#dict 반복문에서 key만 뽑고자 하면, for key in by_rating
#dict 반복문에서 value만 뽑고자 하면, for val in by_rating.values()
for _, item_list in sorted(by_rating.items()): 
    n_total = len(item_list)
    n_subset = int(args.proportion_subset_of_train * n_total)
    review_subset.extend(item_list[:n_subset]) # 0부터 27999까지 따로 review_subset으로 담는다

In [17]:
print(n_total, n_subset)

280000 28000


In [20]:
review_subset[0:5]

[{'rating': 1,
  'review': "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars."},
 {'rating': 1,
  'review': "I don't know what Dr. Goldberg was like before  moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in the co-pay and having you com

In [21]:
review_subset = pd.DataFrame(review_subset)
review_subset.head()

,rating,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,1,I don't know what Dr. Goldberg was like before...
2,1,I'm writing this review to give you a heads up...
3,1,Wing sauce is like water. Pretty much a lot of...
4,1,Owning a driving range inside the city limits ...


In [22]:
train_reviews.rating.value_counts()

2    280000
1    280000
Name: rating, dtype: int64

In [23]:
# train_reviews에서 딱 10% 담아냄
review_subset.rating.value_counts()

2    28000
1    28000
Name: rating, dtype: int64

In [24]:
#중복제거하고 평가 레이팅 점수 갯수 구함
set(review_subset.rating)

{1, 2}

In [25]:
#28000개 데이터를 가지고 있는 subset을 train, val, test 데이터를 나누는 작업 진행
by_rating = collections.defaultdict(list)
for _, row in review_subset.iterrows():
    by_rating[row.rating].append(row.to_dict())

In [27]:
by_rating[2][:5]

[{'rating': 2,
  'review': "Been going to Dr. Goldberg for over 10 years. I think I was one of his 1st patients when he started at MHMG. He's been great over the years and is really all about the big picture. It is because of him, not my now former gyn Dr. Markoff, that I found out I have fibroids. He explores all options with you and is very patient and understanding. He doesn't judge and asks all the right questions. Very thorough and wants to be kept in the loop on every aspect of your medical health and your life."},
 {'rating': 2,
  'review': 'All the food is great here. But the best thing they have is their wings. Their wings are simply fantastic!!  The \\"Wet Cajun\\" are by the best & most popular.  I also like the seasoned salt wings.  Wing Night is Monday & Wednesday night, $0.75 whole wings!\\n\\nThe dining area is nice. Very family friendly! The bar is very nice is well.  This place is truly a Yinzer\'s dream!!  \\"Pittsburgh Dad\\" would love this place n\'at!!'},
 {'ratin

In [28]:
final_list = []
np.random.seed(args.seed)

In [30]:
for _, item_list in sorted(by_rating.items()):
    np.random.shuffle(item_list)# 데이터 쪼개기 전에 한번 섞어주고

    n_total = len(item_list)
    #각각 정한 포션별로 데이터 분배
    n_train = int(args.train_proportion * n_total)
    n_val = int(args.val_proportion * n_total)
    n_test = int(args.test_proportion * n_total)

    #데이터 포인트에게 split attribute 부여
    for item in item_list[:n_train]:
        item['split'] = 'train'
    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'
    for item in item_list[n_train+n_val:n_train+n_val+n_test]:
        item['split'] = 'test'

    final_list.extend(item_list)

In [36]:
final_list[9999], final_list[20000], final_list[27000]

({'rating': 1,
  'review': 'Place is avg.  Sushi was good but the red sangria saki was bad. Service was sporadic and we had a one plate that they forgot to put the sauce on, and another couple plates they tried to give us which we did not order.   I would try to find another place.',
  'split': 'train'},
 {'rating': 1,
  'review': "I'm only giving one star because I have too. I have been coming to this car wash for years and I've always thought highly of this place and the owner. I referred my good friend to this place for a mini detail, and payed for it myself to show him how good this place is. When he picked up his car they jammed his key in the ignition and could not get it out. They took no responsibility for it and the owner told us it's not her problem. Then when we took it to honda to get fixed, they informed us they got a sensor wet underneath the shifting council and now that needs to be replaced as well. I am definitely taking them to court.",
  'split': 'val'},
 {'rating': 

In [44]:
final_reviews = pd.DataFrame(final_list)

In [45]:
final_reviews.split.value_counts()

train    39200
test      8400
val       8400
Name: split, dtype: int64

전처리 함수를 이렇게 데이터에 적용시키는 건 처음 봄. 이렇게도 되는군

In [46]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

final_reviews.review = final_reviews.review.apply(preprocess_text)

In [47]:
final_reviews[0:10]

,rating,review,split
0,1,ordered my usual peanut butter cup smoothie . ...,train
1,1,i love a good chicken sandwich and ordered two...,train
2,1,don t think i ll be back . i just get the feel...,train
3,1,steak was cooked good but even though it was a...,train
4,1,"the concept of this place is pretty neat , but...",train
5,1,this was for carry out . the women up front we...,train
6,1,"was taken here for moms day , we got right in ...",train
7,1,yuck ! n nit s almost embarrassing having this...,train
8,1,don t bother if you care about customer servic...,train
9,1,"really , the only redeeming quality of this ga...",train


In [48]:
final_reviews['rating'] = final_reviews.rating.apply({1:'negative', 2:'positive'}.get)

In [49]:
final_reviews.head()

,rating,review,split
0,negative,ordered my usual peanut butter cup smoothie . ...,train
1,negative,i love a good chicken sandwich and ordered two...,train
2,negative,don t think i ll be back . i just get the feel...,train
3,negative,steak was cooked good but even though it was a...,train
4,negative,"the concept of this place is pretty neat , but...",train


In [52]:
# final_reviews 파일 내보내기
final_reviews.to_csv(args.output_munged_csv, index=False)